https://adventofcode.com/2023/day/11

In [53]:
import itertools as it

import numpy as np

In [109]:
with open("data/11.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
"""

In [9]:
def parse_input(data):
    return np.array(
        [[1 if c == "#" else 0 for c in line] for line in data.splitlines()]
    )


def embiggen(image):
    zh = np.zeros_like(image[0])
    vblocks = np.vsplit(image, np.nonzero(image.sum(axis=1) == 0)[0])
    taller = np.vstack(
        list(it.chain.from_iterable(it.zip_longest(vblocks, [], fillvalue=zh)))[:-1]
    )

    zv = np.zeros_like(taller[:, :1])
    hblocks = np.hsplit(taller, np.nonzero(taller.sum(axis=0) == 0)[0])
    wider = np.hstack(
        list(it.chain.from_iterable(it.zip_longest(hblocks, [], fillvalue=zv)))[:-1]
    )
    return wider


def sum_shortest_paths(image):
    points = zip(*np.nonzero(image))
    combos = it.combinations(points, 2)
    return sum(manhattan(a, b) for (a, b) in combos)


def manhattan(a, b):
    a1, a2 = a
    b1, b2 = b
    return abs(a1 - b1) + abs(a2 - b2)

In [107]:
sum_shortest_paths(embiggen(parse_input(testdata)))

374

In [110]:
sum_shortest_paths(embiggen(parse_input(data)))

9947476

### Part 2
So much for numpy array manipulation.

In [227]:
def parse_input_2(data):
    return [
        (r, c)
        for r, line in enumerate(data.splitlines())
        for c, char in enumerate(line)
        if char == "#"
    ]


def contiguous_intervals(ar):
    return [
        (x[0], x[-1] + 1)
        for x in np.split(ar, np.nonzero(np.append(np.array([0]), np.diff(ar)) > 1)[0])
    ]


def expand_cosmos(pts, factor=2):
    rci = contiguous_intervals(sorted({r for (r, c) in pts}))
    cci = contiguous_intervals(sorted({c for (r, c) in pts}))

    for a, b in list(zip(rci[:-1], rci[1:]))[::-1]:
        gap = b[0] - a[1]
        offset = gap * (factor - 1)
        pts = [
            (r + offset if r >= b[0] else r, c) for r, c in pts
        ]
    
    for a, b in list(zip(cci[:-1], cci[1:]))[::-1]:
        gap = b[0] - a[1]
        offset = gap * (factor - 1)
        pts = [
            (r, c + offset if c >= b[0] else c) for r, c in pts
        ]

    return pts


def manhattan(a, b):
    a1, a2 = a
    b1, b2 = b
    return abs(a1 - b1) + abs(a2 - b2)


def sum_expanded_cosmos_distances(data, factor=2):
    pts = parse_input_2(data)
    expanded_pts = expand_cosmos(pts, factor=factor)
    return sum(manhattan(a, b) for a, b in it.combinations(expanded_pts, 2))
    

In [228]:
sum_expanded_cosmos_distances(testdata, factor=2)

374

In [229]:
sum_expanded_cosmos_distances(data, factor=2)

9947476

In [230]:
sum_expanded_cosmos_distances(testdata, factor=10)

1030

In [231]:
sum_expanded_cosmos_distances(testdata, factor=100)

8410

In [232]:
sum_expanded_cosmos_distances(data, factor=1_000_000)

519939907614